In [1]:
import ebooklib
from ebooklib import epub
import re
import os

In [2]:
def read_epub_book(epub_file, ID):
    book = epub.read_epub(epub_file)
    text = ""

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)
            content = re.sub('\s+', ' ', content)
            content = re.sub('\n', ' ', content)
            content = re.sub("&#13;", ' ', content)
            text += content
            
    book_item = {'text': text, 'bookID':ID}
    return book_item

In [3]:
read_epub_book('C:\\Users\\user\\Desktop\\more-utopia.epub', 'UTOPIA')

F:\Anaconda\envs\py38\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


{'text': ' Utopia Table of Contents Introduction Discourses of Raphael Hythloday, of the Best State of a Commonwealth Of Their Towns, Particularly of Amaurot Of Their Magistrates Of Their Trades, and Manner of Life Of Their Traffic Of the Travelling of the Utopians Of Their Slaves, and of Their Marriages Of Their Military Discipline Of the Religions of the Utopians   Utopia Saint Thomas More Copyright © 2014 epubBooks All Rights Reserved. This publication is protected by copyright. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epu

## Split

In [5]:
def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

In [6]:
def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [7]:
read_epub_paragraphs('more-utopia.epub',1)

[{'paragraph': 'Utopia Saint Thomas More Copyright © 2014 epubBooks All Rights Reserved. This publication is protected by copyright. By payment of the required fees, you have been granted the non-exclusive, non-transferable right to access and read the text of this ebook on-screen or via personal text-to-speech computer systems. No part of this text may be reproduced, transmitted, downloaded, decompiled, reverse engineered, stored in or introduced into any information storage and retrieval system, in any form or by any means, whether electronic or mechanical, now known or hereinafter invented, without the express written permission of epubBooks. www.epubbooks.com First published in 1901 by Cassell &amp; Company',
  'nr': 1,
  'bookID': 1},
 {'paragraph': "Introduction Sir Thomas More, son of Sir John More, a justice of the King's Bench, was born in 1478, in Milk Street, in the city of London. After his earlier education at St. Anthony's School, in Threadneedle Street, he was placed, as

In [15]:
#!pip install pyenchant
#!pip install --no-binary pyenchant

In [16]:
#!apt-get install -y libenchant1c2a
#!apt-get update

In [17]:
#!apt-get install libenchant-2-2

In [18]:
import enchant

In [19]:
ENGLISH_DICT1 = enchant.Dict("en_UK")
ENGLISH_DICT2 = enchant.Dict("en_US")

In [20]:
def is_english_word(word):
    return (ENGLISH_DICT1.check(word) or ENGLISH_DICT2.check(word))

In [21]:
is_english_word('test')

True

## TF-IDF

In [22]:
#!pip install gensim

In [23]:
#!pip install nltk

In [25]:
paragraphs = read_epub_paragraphs('more-utopia.epub', 'UTOPIA')

F:\Anaconda\envs\py38\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [26]:
paragraph = paragraphs[66]['paragraph']

In [27]:
paragraph

" One day, when I was dining with him, there happened to be at table one of the English lawyers, who took occasion to run out in a high commendation of the severe execution of justice upon thieves, 'who,' as he said, 'were then hanged so fast that there were sometimes twenty on one gibbet!' and, upon that, he said, 'he could not wonder enough how it came to pass that, since so few escaped, there were yet so many thieves left, who were still robbing in all places."

### 

### The Function

In [ ]:
import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(paragraphs):
    processed_paragraphs = []
    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word not in stop_words]
        stemmed_words = [stemmer.stem(word) for word in filtered_words]
        processed_paragraphs.append(" ".join(stemmed_words))
    return processed_paragraphs

In [36]:
processedDoc = preprocess([p['paragraph'] for p in paragraphs])

In [37]:
processedDoc

['utopia saint thoma copyright epubbook right reserv public protect copyright payment requir fee grant non exclus non transfer right access read text ebook screen via person text speech comput system part text may reproduc transmit download decompil revers engin store introduc inform storag retriev system form mean whether electron mechan known hereinaft invent without express written permiss epubbook www epubbook com first publish cassel amp compani',
 'introduct sir thoma son sir john justic king bench wa born milk street citi london earlier educ anthoni school threadneedl street wa place boy household cardin john morton archbishop canterburi lord chancellor',
 'wa unusu person wealth influenc son good famili establish togeth relat patron client youth wore patron liveri ad state',
 'patron use afterward wealth influenc help young client forward world cardin morton earlier day bishop eli richard iii sent tower wa busi afterward hostil richard wa chief advis henri vii',
 'made archbish

# 

## Vectorize

In [39]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/52/2d/ad6928a578c78bb0e44e34a5a922818b14c56716b81d145924f1f291416f/scikit_learn-1.3.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.3 MB 1.6 MB/s eta 0:00:06
   --- ------------------------------------ 0.8/9.3 MB 10.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.0/9.3 MB 15.7 MB/s eta 0:00:01
   -------------- ------------------------- 3.3/9.3 MB 19.0 MB/s eta 0:00:01
   ------------------- -------------------- 4.6/9.3 MB 20.9 MB/s eta 0:00:01
   ------------------------- -------------- 5.9/9.3 MB 22.3 MB/s eta 0:00:01
   ------------------------------- -

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
vectorizer = TfidfVectorizer(min_df=2)

In [50]:
tfidf_matrix = vectorizer.fit_transform(processedDoc)

In [51]:
tfidf_matrix

<456x1656 sparse matrix of type '<class 'numpy.float64'>'
	with 13180 stored elements in Compressed Sparse Row format>

In [52]:
len(processedDoc)

456

In [53]:
vectorizer.get_feature_names_out()

array(['abandon', 'abl', 'abound', ..., 'younger', 'youth', 'zeal'],
      dtype=object)

In [54]:
len(vectorizer.get_feature_names_out())

1656

## Search

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
query = 'city by the people'
processedQuery = preprocess([query])[0]
print(processedQuery)

citi peopl


In [59]:
processedQuery = 'citi peopl'

In [60]:
query_vector = vectorizer.transform([processedQuery])

In [61]:
query_vector

<1x1656 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [62]:
similarities = cosine_similarity(query_vector, tfidf_matrix)

In [63]:
similarities

array([[0.        , 0.12421983, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.17807528, 0.        ,
        0.1439686 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.06862567, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.07103544, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05209394, 0.  

In [64]:
len(similarities[0])

456

In [65]:
nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

In [66]:
nearest_neighbor_index, similarity_score

(278, 0.2871506953618768)

In [70]:
paragraphs[278]

{'paragraph': " If any man has a mind to travel only over the precinct of his own city, he may freely do it, with his father's permission and his wife's consent; but when he comes into any of the country houses, if he expects to be entertained by them, he must labour with them and conform to their rules; and if he does this, he may freely go over the whole precinct, being then as useful to the city to which he belongs as if he were still within it.",
 'nr': 309,
 'bookID': 'UTOPIA'}

# 

### The Function

In [73]:
from sklearn.metrics.pairwise import cosine_similarity

query = 'city by the people'
processedQuery = preprocess([query])[0]
query_vector = vectorizer.transform([processedQuery])

similarities = cosine_similarity(query_vector, tfidf_matrix)

num_top_results = 10
top_indices = similarities.argsort()[0, ::-1][:num_top_results]

similar_paragraphs = [paragraphs[i] for i in top_indices]

for i, paragraph_index in enumerate(top_indices):
    similarity_score = similarities[0, paragraph_index]
    print(f"Similarity Score of the Output: Paragraph {i+1}: {similarity_score}")
    print(f"Paragraph {i+1}: {paragraphs[paragraph_index]}")
    print()

Similarity Score of the Output: Paragraph 1: 0.2871506953618768
Paragraph 1: {'paragraph': " If any man has a mind to travel only over the precinct of his own city, he may freely do it, with his father's permission and his wife's consent; but when he comes into any of the country houses, if he expects to be entertained by them, he must labour with them and conform to their rules; and if he does this, he may freely go over the whole precinct, being then as useful to the city to which he belongs as if he were still within it.", 'nr': 309, 'bookID': 'UTOPIA'}

Similarity Score of the Output: Paragraph 2: 0.27499489185233733
Paragraph 2: {'paragraph': ' When they draw out troops of their own people, they take such out of every city as freely offer themselves, for none are forced to go against their wills, since they think that if any man is pressed that wants courage, he will not only act faintly, but by his cowardice dishearten others.', 'nr': 490, 'bookID': 'UTOPIA'}

Similarity Score of

# 

## Dimention Reduction

In [75]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [76]:
n_components = 250
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)

In [77]:
reduced_query_vec = svd.transform(query_vector)

In [78]:
len(reduced_query_vec[0])

250

In [79]:
similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)

In [80]:
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]

In [81]:
nearest_neighbor_index

437

In [82]:
paragraphs[437]

{'paragraph': ' When they draw out troops of their own people, they take such out of every city as freely offer themselves, for none are forced to go against their wills, since they think that if any man is pressed that wants courage, he will not only act faintly, but by his cowardice dishearten others.',
 'nr': 490,
 'bookID': 'UTOPIA'}

# 

### The Function

In [83]:
#import numpy as np
#from scipy.sparse import random
#from sklearn.decomposition import TruncatedSVD
#from sklearn.metrics.pairwise import cosine_similarity

#def find_similar_paragraphs(query, tfidf_matrix, vectorizer, paragraphs, svd=None, n_components=250, num_top_results=5):
    #processed_query = preprocess([query])[0]
    #query_vector = vectorizer.transform([processed_query])
    
    #if svd is not None:
        #query_vector = svd.transform(query_vector)
    
    #if svd is None:
        #similarities = cosine_similarity(query_vector, tfidf_matrix)
    #else:
        #reduced_matrix = svd.transform(tfidf_matrix)
        #similarities = cosine_similarity(query_vector, reduced_matrix)
    
    #top_indices = similarities.argsort()[0, ::-1][:num_top_results]
    
    #similar_paragraphs = [paragraphs[i] for i in top_indices]
    
    #for i, paragraph_index in enumerate(top_indices):
        #similarity_score = similarities[0, paragraph_index]
        #print(f"Similarity Score for Paragraph {i+1}: {similarity_score}")
        #print(f"Paragraph {i+1}: {paragraphs[paragraph_index]}")
        #print()
    
    #return similar_paragraphs

In [84]:
#similar_paragraphs = find_similar_paragraphs(query='city by the people',
                                             #tfidf_matrix=tfidf_matrix,
                                             #vectorizer=vectorizer,
                                             #paragraphs=paragraphs,
                                             #svd=svd,
                                             #n_components=250,
                                             #num_top_results=5)

# 

## Vectorize

In [85]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 

### The Function

In [87]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim

def preprocess2(paragraphs):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    processed_paragraphs = []
    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        filtered_words = [word for word in lemmatized_words if word not in stop_words]
        processed_paragraphs.append(filtered_words)
    return processed_paragraphs

texts = preprocess2([p['paragraph'] for p in paragraphs])
texts

[['utopia',
  'saint',
  'thomas',
  'copyright',
  'epubbooks',
  'right',
  'reserved',
  'publication',
  'protected',
  'copyright',
  'payment',
  'required',
  'fee',
  'granted',
  'non',
  'exclusive',
  'non',
  'transferable',
  'right',
  'access',
  'read',
  'text',
  'ebook',
  'screen',
  'via',
  'personal',
  'text',
  'speech',
  'computer',
  'system',
  'part',
  'text',
  'may',
  'reproduced',
  'transmitted',
  'downloaded',
  'decompiled',
  'reverse',
  'engineered',
  'stored',
  'introduced',
  'information',
  'storage',
  'retrieval',
  'system',
  'form',
  'mean',
  'whether',
  'electronic',
  'mechanical',
  'known',
  'hereinafter',
  'invented',
  'without',
  'express',
  'written',
  'permission',
  'epubbooks',
  'www',
  'epubbooks',
  'com',
  'first',
  'published',
  'cassell',
  'amp',
  'company'],
 ['introduction',
  'sir',
  'thomas',
  'son',
  'sir',
  'john',
  'justice',
  'king',
  'bench',
  'wa',
  'born',
  'milk',
  'street',
  'ci

In [90]:
texts = preprocess2([p['paragraph'] for p in paragraphs])
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(vector_size=250, min_count=0, alpha=0.025, min_alpha=0.025, epochs=100)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

test = model.predict_output_word(['people', 'organic', 'rural', 'city'])
print(test)

query = 'organic city by people'
tokens = preprocess2([query])[0]
vector = model.infer_vector(tokens)
print(tokens)
print(vector)

similar_paragraphs = model.dv.most_similar(positive=[vector])
print(similar_paragraphs)

[('city', 0.9809526), ('take', 0.008571321), ('four', 0.0023320194), ('pay', 0.0015958901), ('divided', 0.0008432916), ('mutual', 0.0007751519), ('lie', 0.0006675498), ('commerce', 0.0005085444), ('live', 0.00035953222), ('enslave', 0.0003365525)]
['organic', 'city', 'people']
[-5.57341874e-02  1.59105554e-01  1.74876884e-01  1.57169580e-01
  3.58289331e-02  2.55414069e-01  1.82126220e-02  5.51910162e-01
 -4.18453515e-02  2.36901388e-01  1.49862751e-01 -5.95481694e-02
 -2.33165890e-01  6.63203076e-02 -1.22977130e-01 -9.23415646e-02
 -4.51102376e-01  1.22466289e-01 -1.60397533e-02 -7.10813105e-02
 -5.46913624e-01 -2.07856327e-01  3.47226381e-01  2.31825560e-01
  3.72686833e-01 -5.66952005e-02  3.56794387e-01 -2.88631141e-01
 -1.74229816e-01 -8.80672187e-02  1.41924005e-02  3.82537007e-01
 -1.33885309e-01 -3.28339994e-01  8.41183588e-02 -6.15718327e-02
  6.73531741e-02 -4.98795599e-01 -1.06212750e-01 -2.80644000e-01
 -4.05796152e-03 -1.23456925e-01  4.66116183e-02  3.46350908e-01
  2.818

In [91]:
paragraphs[278]

{'paragraph': " If any man has a mind to travel only over the precinct of his own city, he may freely do it, with his father's permission and his wife's consent; but when he comes into any of the country houses, if he expects to be entertained by them, he must labour with them and conform to their rules; and if he does this, he may freely go over the whole precinct, being then as useful to the city to which he belongs as if he were still within it.",
 'nr': 309,
 'bookID': 'UTOPIA'}

# 

### The Function

In [92]:
def Match(sentence, K, model):
    tokens = preprocess2([sentence])[0]
    
    vector = model.infer_vector(tokens)
    
    similarities = model.dv.most_similar(positive=[vector], topn=K)
    
    return similarities

In [96]:
K = 10
sentence = 'organic urban city built by self reliance people'
best_matches = Match(sentence, K, model)

for i, match in enumerate(best_matches, 1):
    print(f"{i}. Similarity: {match[1]}, Paragraph: {paragraphs[match[0]]}")

1. Similarity: 0.6179856061935425, Paragraph: {'paragraph': 'Of Their Traffic "But it is now time to explain to you the mutual intercourse of this people, their commerce, and the rules by which all things are distributed among them. "As their cities are composed of families, so their families are made up of those that are nearly related to one another.', 'nr': 277, 'bookID': 'UTOPIA'}
2. Similarity: 0.4969038665294647, Paragraph: {'paragraph': ' They have built, over all the country, farmhouses for husbandmen, which are well contrived, and furnished with all things necessary for country labour. Inhabitants are sent, by turns, from the cities to dwell in them; no country family has fewer than forty men and women in it, besides two slaves.', 'nr': 213, 'bookID': 'UTOPIA'}
3. Similarity: 0.4797016978263855, Paragraph: {'paragraph': ' Even the Syphogrants, though excused by the law, yet do not excuse themselves, but work, that by their examples they may excite the industry of the rest of t

# 

## PDF Extracting

In [97]:
import PyPDF2
import fitz

In [98]:
pdf_file_path = "C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print.pdf"
pdf_document = fitz.open(pdf_file_path)
page = pdf_document.load_page(9)
textInfo = page.get_text('blocks',flags=1+2+8)

In [99]:
text = [t[4] for t in textInfo]

In [100]:
text

['| RC 11\n 11 |  2022 - 2023 | City in Reverie | \n',
 'LITERATURE\n',
 'In Orhan Pamuk’s “The Museum of Innocence,” Kemal Basmaci, a wealthy Istanbul \n',
 'businessman, falls in love with his distant cousin, Fusun. The novel revolves \n',
 'around a physical museum that Kemal creates to display objects associated with \n',
 'his relationship with Fusun. These objects provide a glimpse into the characters’ \n',
 'lives, thoughts, and emotions.\n',
 'Exploring these objects provides insight into the complexities of human emotion \n',
 'and the importance of cherishing our memories. It encourages people to reflect \n',
 'on the value of their own experiences and the memories that make up their lives. \n']

# 

In [101]:
def extract_text_from_pdf(pdf_path):
    text = {}
    pdf_document = fitz.open(pdf_path)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        textInfo = page.get_text("blocks", flags=1+2+8)
        paragraphs = [re.sub('\n', ' ', t[4]) for t in textInfo]
        text[page_num] = paragraphs
    pdf_document.close()
    return text

def extract_images_from_pdf(pdf_path, image_output_dir):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        image_list = page.get_images(full=True)

        for img_index, img_info in enumerate(image_list):
            image = pdf_document.extract_image(img_info[0])
            image_path = f"{image_output_dir}/page_{page_num + 1}_image_{img_index}.png"
            with open(image_path, "wb") as image_file:
                image_file.write(image["image"])

            images.append(image_path)

    pdf_document.close()
    return images

In [102]:
pdf_file_path = "C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print.pdf"
output_text = extract_text_from_pdf(pdf_file_path)

In [103]:
output_text

{0: [],
 1: ['CONTENTS . ', 'WHAT? WHY? ', '01 02 ', '06 10 '],
 2: ['HOW? WITH WHAT? NEXT ', '03 04 05 ', '16 50 78 '],
 3: ['WHAT? 01 ',
  'Istanbul’s mesmerizing allure and rich cultural heritage are a reflection of its  ',
  'diverse history and captivating charm. As the city evolves and grows, it is imperative  ',
  'to preserve its essence and honour its past while creating a promising future. Its  ',
  'streets are adorned with a plethora of objects, each holding a unique story that  ',
  'contributes to the city’s vibrant tapestry. '],
 4: [],
 5: ['| RC 11  11 |  2022 - 2023 | City in Reverie |  ',
  'INTRODUCTION ',
  'Identity of the city | Abstract urban elements as fundamental objects  ',
  'Every place, to some extent, can be remembered partly because it is unique,  ',
  'but partly because it has affected our consciousness and generated enough  ',
  'association to hold in it our personal worlds. Human bodies, serving as a bridge  ',
  'between the tangible and visible r

# 

In [104]:
output_image_dir = pdf_file_path.split('.')[0] + '_Images'
os.mkdir(output_image_dir)
extracted_images = extract_images_from_pdf(pdf_file_path, output_image_dir)

In [105]:
extracted_images

['C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_4_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_5_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_6_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_7_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_8_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_9_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_10_image_0.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_10_image_1.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_Print_Images/page_10_image_2.png',
 'C:\\Users\\user\\Desktop\\X\\OneDrive_2_2024-1-20\\City_in_Reverie_P